## Create Multiple Hardware Schemas

When a measurement is associated with multiple hardware entities, it may be desirable for each type of hardware to have its own metadata schema.  This demonstrates how to create multiple hardware entities with different schemas.

In [ ]:
from ni.datastore.metadata import MetadataStoreClient

metadata_store_client = MetadataStoreClient()
# Register schemas
cable_schema_id = metadata_store_client.register_schema_from_file("cable_schema.toml")
socket_schema_id = metadata_store_client.register_schema_from_file("socket_schema.toml")
scope_schema_id = metadata_store_client.register_schema_from_file("scope_schema.toml")
test_result_schema_id = metadata_store_client.register_schema_from_file("test_result_schema.toml")

## Create the Hardware Objects with their Custom Metadata

Now that the schemas are registered, we can create Hardware objects that specify custom metadata to meet the requirements of the registered schemas.  Each piece of hardware can use it's own schema.

In [ ]:
from ni.datastore.metadata import HardwareItem

cable = HardwareItem(
    manufacturer="NI",
    model="cable",
    serial_number="7u2349",
    schema_id=cable_schema_id,
)
cable.extension["cable_length"] = "1.5"
cable.extension["manufacture_date"] = "2023-01-01"

socket = HardwareItem(
    manufacturer="NI",
    model="socket",
    schema_id=socket_schema_id,
)
socket.extension["socket_number"] = "3"
socket.extension["manufacture_date"] = "2024-05-01"

scope = HardwareItem(
    manufacturer="NI",
    model="PXIe-5171",
    serial_number="1933B4E",
    schema_id=scope_schema_id,
)
scope.extension["bandwidth"] = "250 MHz"
scope.extension["manufacture_date"] = "2024-11-03"

## Create a Schema for the Test Result (optional)

If you specify a schema for the test result, it will ensure that all the hardware custom metadata is a superset of whatever metadata is specified in the session schema.  This allows the author to ensure all hardware has some consistent attributes.


In [ ]:
test_result_schema_id = metadata_store_client.register_schema_from_file("test_result_schema.toml")

## Publish a Measurement with the Hardware

Now that all the schemas have been registered and the hardware objects have been created, we can publish our data with the associated hardware.

In [ ]:
from datetime import timezone
import hightime as ht
from ni.datastore.data import DataStoreClient, TestResult, Step
from nitypes.waveform import AnalogWaveform
from nitypes.waveform import Timing
import numpy as np

cable_id = metadata_store_client.create_hardware_item(hardware_item=cable)
socket_id = metadata_store_client.create_hardware_item(hardware_item=socket)
scope_id = metadata_store_client.create_hardware_item(hardware_item=scope)

data_store_client = DataStoreClient()
test_result_id = data_store_client.create_test_result(
    test_result=TestResult(
        name="scope measurements",
        hardware_item_ids=[cable_id, socket_id, scope_id],
        schema_id=test_result_schema_id
    )
)

waveform = AnalogWaveform(
    sample_count=3,
    raw_data=np.array([1.0, 2.0, 3.0]),
    timing=Timing.create_with_regular_interval(
        ht.timedelta(seconds=1e-3),
        ht.datetime.now(timezone.utc)
    )
)

step = Step(name="Initial step", test_result_id=test_result_id)
step_id = data_store_client.create_step(step)
data_store_client.publish_measurement(
    name="scope reading",
    value=waveform,
    step_id=step_id,
)

## Query for the Published Data

Now that we've published some data, we can use our OData query API to find it

In [ ]:
published_measurements = data_store_client.query_measurements("$filter=name eq 'scope reading'")
found_measurement = next(iter(published_measurements), None)
if found_measurement is not None:
    waveform = data_store_client.read_data(found_measurement, expected_type=AnalogWaveform)
    print(f"published data is: {waveform.raw_data}")

## Close the Clients

In [ ]:
metadata_store_client.close()
data_store_client.close()